<a href="https://colab.research.google.com/github/totvslabs/carol-notebooks/blob/main/notebooks/TenantCleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tenant Cleanup
`This notebook will delete given datamodels also deleting its relationships and subscriptions. This script runs asynchronously so the deletion process of multiple datamodels can be faster.`
`The script will ask for the following json on execution:`

```python
{
    "authentication_config" : {
        "username": "username@totvs.com.br",
        "password": "password",
        "organization": "YourOrganization",
        "tenantName": "YourTenantName"
    },
    "script_config" : {
        "environments" : {
            "organization1": [
                "tenantName1",
                "tenantName2",
                "tenantName3"
            ],
            "organization2": [
                "tenantName4",
                "tenantName5",
                "tenantName6"
            ]
        },
        "datamodels" : [
            "dmname1",
            "dmname2",
            "dmname3",
            "dmname4"
        ]
    }
}
```

#### REQUIREMENTS
`These are the packages the script needs before execution.`

In [37]:
%%capture
!pip install --quiet pycarol
from pycarol import PwdAuth, Carol
from google.colab import files
from multiprocessing.dummy import Pool
import sys
import json
import requests

#### CAROL LOGIN FUNCTIONS
`These are the functions made to login into Carol. They will be the same for all notebooks (ideally) and will use pyCarol.` 

[pyCarol reference](https://github.com/totvslabs/pyCarol)

In [38]:
def carol_connect(username, password, organization, tenantName):
    print(f"Connecting to Carol tenant {tenantName}... ", end="\n")

    return Carol(domain=tenantName,
                auth=PwdAuth(username, password), organization=organization)

#### SCRIPT FUNCTIONS
`If the script requires more functions to execute, they will be here.`

In [39]:
def carol_call_api(method="GET", uri="/", headers=None, payload=None, forward=False):
    endpoint = f"https://api.carol.ai{uri}"

    response = requests.request(
        method, endpoint, headers=headers, data=payload)

    if forward:
        return response

    if response.status_code != 200:
        raise ValueError(
            f"Request error {response.status_code}. Message: \n{response.text}"
        )

    return response.json()


def carol_token(carol: dict) -> dict:
    uri = "/api/v1/oauth2/token"

    payload = {"grant_type": "password", **carol}
    headers = {
        "accept": "application/json",
        "Content-Type": "application/x-www-form-urlencoded",
    }

    return carol_call_api("POST", uri, headers, payload)


def carol_refresh_token(carol: dict) -> dict:
    uri = "/api/v1/oauth2/token"

    payload = {
        "grant_type": "refresh_token",
        "refresh_token": carol["refresh_token"]
    }
    headers = {
        "accept": "application/json",
        "Content-Type": "application/x-www-form-urlencoded",
    }

    return carol_call_api("POST", uri, headers, payload)


def carol_validate_token(carol: dict) -> dict:
    uri = f"/api/v1/oauth2/token/{carol['access_token']}"

    payload = {}
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }

    auth = None
    response = carol_call_api("GET", uri, headers, payload, forward=True)

    if response.status_code == 401:
        auth = carol_refresh_token(carol)
    else:
        auth = response.json()

    return {**carol, **auth}


def carol_connect(username, password, tenant_name) -> dict:
    print(f"Connecting to Carol tenant {tenant_name}...", end=" ")

    carol = {
        "connectorId": "0a0829172fc2433c9aa26460c31b78f0",
        "username": username,
        "password": password,
        "subdomain": tenant_name,
    }

    auth = carol_token(carol)

    print("OK")
    return {**carol, **auth}


def on_request_success(response: requests.Response):
    if response.status_code == 200:
        print(f'Success: {response.json()["success"]}')
    else:
        print(f'Failed: {response.status_code} - {response.json()["message"]}')


def on_request_error(ex: Exception):
    print(f'Request failed: {ex}')


def carol_call_api_async(method="GET", uri="/", headers=None, payload=None, pool=None):
    endpoint = f"https://api.carol.ai{uri}"

    pool.apply_async(requests.request, args=[method, endpoint],
                     kwds={'headers': headers, 'data': payload}, callback=on_request_success, error_callback=on_request_error)

    pass

def remove_datamodel(carol, name) -> dict:
    uri = f"/api/v3/entities/templates/name/{name}?entitySpace=PRODUCTION"

    payload = {}
    headers = {
        "Authorization": carol["access_token"],
        "accept": "application/json",
    }

    try:
        datamodel = carol_call_api("GET", uri, headers, payload)
        mdmId = datamodel["mdmId"]
    except ValueError:
        error = str(sys.exc_info()[1])
        return {"success":error}

    if mdmId:
        uri = f"/api/v1/entities/templates/{mdmId}?entitySpace=PRODUCTION"
        payload = {}
        headers = {
            "Authorization": carol["access_token"],
            "accept": "application/json",
        }

        try:
            deleted = carol_call_api("DELETE", uri, headers, payload)
            return deleted
        except ValueError:
            error = str(sys.exc_info()[1])
            return {"success":error}



def remove_datamodel_async(carol, name, pool) -> dict:
    uri = f"/api/v3/entities/templates/name/{name}?entitySpace=PRODUCTION"

    payload = {}
    headers = {
        "Authorization": carol["access_token"],
        "accept": "application/json",
    }

    try:
        datamodel = carol_call_api("GET", uri, headers, payload)
        mdmId = datamodel["mdmId"]
    except ValueError:
        error = str(sys.exc_info()[1])
        return {"success":error}

    if mdmId:
        uri = f"/api/v1/entities/templates/{mdmId}?entitySpace=PRODUCTION"
        payload = {}
        headers = {
            "Authorization": carol["access_token"],
            "accept": "application/json",
        }

        try:
            deleted = carol_call_api_async("DELETE", uri, headers, payload, pool)
            return deleted
        except ValueError:
            error = str(sys.exc_info()[1])
            return {"success":error}


def remove_datamodel_relationships(carol, name) -> dict:
    uri = f"/api/v3/relationship/published?entityTypes={name}&offset=0&pageSize=-1"

    payload = {}
    headers = {
        "Authorization": carol["access_token"],
        "accept": "application/json",
    }

    try:
        response = carol_call_api("GET", uri, headers, payload)
        relationships = response["hits"]
    except ValueError:
        error = str(sys.exc_info()[1])
        return {"success":error}
    
    if relationships:
        payload = {}
        headers = {
            "Authorization": carol["access_token"],
            "accept": "application/json",
        }

        for rel in relationships:
            uri = f"/api/v1/relationship/mapping/{rel['mdmId']}?entitySpace=PRODUCTION"
            carol_call_api("DELETE", uri, headers, payload)

    return {"success":True}


def remove_datamodel_subscriptions(carol, name) -> dict:
    uri = f"/api/v3/subscription/template/{name}"

    payload = {}
    headers = {
        "Authorization": carol["access_token"],
        "accept": "application/json",
    }

    try:
        subscriptions = carol_call_api("GET", uri, headers, payload)
    except ValueError:
        error = str(sys.exc_info()[1])
        return {"success":error}
    
    if subscriptions:
        payload = {}
        headers = {
            "Authorization": carol["access_token"],
            "accept": "application/json",
        }

        for rel in subscriptions:
            uri = f"/api/v1/subscription/{rel['mdmId']}"
            carol_call_api("DELETE", uri, headers, payload)

    return {"success":True}


def publish_datamodel(carol, name) -> dict:
    uri = f"/api/v3/entities/templates/name/{name}?entitySpace=PRODUCTION"

    payload = {}
    headers = {
        "Authorization": carol["access_token"],
        "accept": "application/json",
    }

    try:
        datamodel = carol_call_api("GET", uri, headers, payload)
        mdmId = datamodel["mdmId"]
    except ValueError:
        error = str(sys.exc_info()[1])
        return {"success":error}

    if mdmId:
        uri = f"/api/v1/entities/templates/{mdmId}/publish"
        payload = {}
        headers = {
            "Authorization": carol["access_token"],
            "accept": "application/json",
        }

        try:
            published = carol_call_api("POST", uri, headers, payload)
            return published
        except ValueError:
            error = str(sys.exc_info()[1])
            return {"success":error}

#### CONFIGURATION FILE
`Now you will need to upload the configuration file with the format given above.`

In [40]:
try:
    config_file = files.upload()
    config_json = json.loads(config_file[next(iter(config_file))].decode("utf-8"))
except:
    with open('./carol.json') as config_file:
        config_json = json.loads(config_file.read())
    config_file.close()

Saving carol.json to carol (6).json


#### SCRIPT EXECUTION
`The main execution of the script will happen here.`

In [41]:
pool = Pool(len(config_json['script_config']['datamodels']))

for org, tenants in config_json['script_config']['environments'].items():
    for index, tenant in enumerate(tenants, start=1):
        carol = carol_connect(config_json['authentication_config']["username"], config_json['authentication_config']["password"], tenant)
        for dm in config_json['script_config']['datamodels']:
            print(f"{tenant} -> {dm} ... ")
            r = remove_datamodel_relationships(carol, dm)
            print(r['success'])
            s = remove_datamodel_subscriptions(carol, dm)
            print(s['success'])
            p = publish_datamodel(carol, dm)
            print(p['success'])

    for index, tenant in enumerate(tenants, start=1):
        carol = carol_connect(config_json['authentication_config']["username"], config_json['authentication_config']["password"], tenant)
        for dm in config_json['script_config']['datamodels']:
            print(f"{tenant} -> {dm} ... ")
            remove_datamodel_async(carol, dm, pool)
            
pool.close()

Connecting to Carol tenant brenopapaunif... OK
brenopapaunif -> customstaging ... 
True
True
False
Connecting to Carol tenant brenopapaunif... OK
brenopapaunif -> customstaging ... 
